# HE4903 Problem Set 1 Question 3

SW (4e) Empirical Exercises E12.1 (part e for Problem Set 2)

How does fertility affect labor supply? That is, how much does a woman’s labor supply fall when she has an additional child? In this exercise, you will estimate this effect using data for married women from the 1980 U.S. Census. The data are available on the text website, http://www.pearsonglobaleditions.com, in the file Fertility and described in the file Fertility_Description. The data set contains information on married women aged 21–35 with two or more children.

In [2]:
# Import relevant libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [3]:
# Open fertility dataset and visualise data
df = pd.read_stata("fertility.dta")
df.head()

,morekids,boy1st,boy2nd,samesex,agem1,black,hispan,othrace,weeksm1
0,0,1,0,0,27,0,0,0,0
1,0,0,1,0,30,0,0,0,30
2,0,1,0,0,27,0,0,0,0
3,0,1,0,0,35,1,0,0,0
4,0,0,0,1,30,0,0,0,22


## a. Regress weeksworked on the indicator variable morekids, using OLS. On average, do women with more than two children work less than women with two children? How much less?

In [4]:
y = df["weeksm1"]
X = sm.add_constant(df["morekids"])
mod = sm.OLS(y, X)
res = mod.fit(cov_type = "HC0", use_t = True)
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                weeksm1   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     3821.
Date:                Mon, 16 Jan 2023   Prob (F-statistic):               0.00
Time:                        18:32:32   Log-Likelihood:            -1.1451e+06
No. Observations:              254654   AIC:                         2.290e+06
Df Residuals:                  254652   BIC:                         2.290e+06
Df Model:                           1                                         
Covariance Type:                  HC0                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.0684      0.056    375.766      0.0

On average, women with more than 2 children worked 5.3870 weeks less than women with less than 2 children.

## b. Explain why the OLS regression estimated in (a) is inappropriate for estimating the causal effect of fertility (morekids) on labor supply (weeksworked).

The OLS regression in (a) is inappropriate for measuring the causal effect of fertility (morekids) on labor supply (weeksm1) because simultaneous causality exists; fertility affects labor supply in that having more kids may be associated with mothers working less, and labor supply may affect fertility in that working less hours may allow mothers to have more kids. Fertility (morekids) is thus an endogenous variable and the reported coefficient is biased and inconsistent.

## c. The data set contains the variable samesex, which is equal to 1 if the first two children are of the same sex (boy–boy or girl–girl) and equal to 0 otherwise. Are couples whose first two children are of the same sex more likely to have a third child? Is the effect large? Is it statistically significant?

In [5]:
y2 = df["morekids"]
X2 = sm.add_constant(df["samesex"])
mod2 = sm.OLS(y2, X2)
res2 = mod2.fit(cov_type = "HC0", use_t = True)
print(res2.summary())

                            OLS Regression Results                            
Dep. Variable:               morekids   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1238.
Date:                Mon, 16 Jan 2023   Prob (F-statistic):          1.38e-270
Time:                        18:32:36   Log-Likelihood:            -1.7673e+05
No. Observations:              254654   AIC:                         3.535e+05
Df Residuals:                  254652   BIC:                         3.535e+05
Df Model:                           1                                         
Covariance Type:                  HC0                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3464      0.001    258.336      0.0

Couples whose first two children are of the same sex are more likely to have more than 2 children. The effect is not large (6.75% more likely) and statistically significant, as evidenced by a low p-value of 0.000.

## d. Explain why samesex is a valid instrument for the IV regression of weeksworked on morekids.

samesex is a valid instrument for morekids as it exhibits instrument relevance, being correlated to morekids (the regressor) and exhibiting instrument exogeneity, being uncorrelated to the error term in the regression framework for weeksm1 because there is no reason to believe that whether a mother's first two children are of the same sex is correlated with the error term for the regression for the number of weeks she works.

## f. Estimate the IV regression of weeksworked on morekids, using samesex as an instrument. How large is the fertility effect on labor supply?

In [6]:
y3 = df["weeksm1"]
X3 = sm.add_constant(df["morekids"])
instrument = sm.add_constant(df["samesex"])

from statsmodels.sandbox.regression.gmm import IV2SLS

mod3 = IV2SLS(y3, X3, instrument = instrument)
res3 = mod3.fit()
print(res3.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                weeksm1   R-squared:                       0.014
Model:                         IV2SLS   Adj. R-squared:                  0.014
Method:                     Two Stage   F-statistic:                     24.54
                        Least Squares   Prob (F-statistic):           7.30e-07
Date:                Mon, 16 Jan 2023                                         
Time:                        18:32:40                                         
No. Observations:              254654                                         
Df Residuals:                  254652                                         
Df Model:                           1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.4211      0.487     43.988      0.0

On average, women with more than 2 children worked 6.3137 weeks less than women with less than 2 children.

## g. Do the results change when you include the variables agem1, black, hispan, and othrace in the labor supply regression (treating these variable as exogenous)? Explain why or why not.

In [7]:
y4 = df["weeksm1"]
X4 = sm.add_constant(df[["morekids", "agem1", "black", "hispan", "othrace"]])
instrument = sm.add_constant(df[["samesex", "agem1", "black", "hispan", "othrace"]])

from statsmodels.sandbox.regression.gmm import IV2SLS

mod4 = IV2SLS(y4, X4, instrument = instrument)
res4 = mod4.fit()
print(res4.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                weeksm1   R-squared:                       0.044
Model:                         IV2SLS   Adj. R-squared:                  0.044
Method:                     Two Stage   F-statistic:                     1335.
                        Least Squares   Prob (F-statistic):               0.00
Date:                Mon, 16 Jan 2023                                         
Time:                        18:32:44                                         
No. Observations:              254654                                         
Df Residuals:                  254648                                         
Df Model:                           5                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.7919      0.407    -11.786      0.0

The results change when the other variables are added as there are more instruments for morekids. This is likely to lead to a more accurate coefficient estimate.

## References

https://www.statsmodels.org/stable/regression.html
https://stackoverflow.com/questions/20701484/why-do-i-get-only-one-parameter-from-a-statsmodels-ols-fit
https://www.kaggle.com/code/carloscinelli/stat-504-robust-standard-errors-python
https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html
https://bfdykstra.github.io/2016/11/17/Endogeneity-and-Instrumental-Variable-Regression.html